In [23]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from bs4 import BeautifulSoup
from tqdm import tqdm
import urllib.parse
import pandas as pd
import re

In [26]:
def get_long_lat(location):
    coords = None
    encoded_location = urllib.parse.quote(location)

    driver.get("https://www.google.com/maps/search/{}".format(encoded_location))
    driver.implicitly_wait(3)
    
    # Get the initial page source
    initial_page_source = driver.page_source
    
    # Wait for the page source to change with a timeout of 10 seconds
    wait = WebDriverWait(driver, 10)
    wait.until(lambda driver: driver.page_source != initial_page_source)

    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")

    meta_tag = soup.find_all("meta", itemprop="image")
    markers = re.split('markers=', str(meta_tag[0]))
    
    if len(markers) > 1:
        x = soup.find('div', class_="qBF1Pd fontHeadlineSmall")
        name_ = re.search('>(.*)<', str(x)).group(1)
        # if name_.lower() == location_name.lower():
        #     coords = re.split('markers=|&amp;', str(meta_tag[0]))[4]
        coords = re.split('markers=|&amp;', str(meta_tag[0]))[4]
    elif len(markers) == 1:
        y = re.split('center=|&amp;', str(meta_tag[0]))
        driver.implicitly_wait(10)
        x = soup.find('h1', class_="DUwDvf lfPIob")
        name_ = re.search('<span class="a5H0ec"></span>(.*?)<span class="G0bp3e"></span>', str(x)).group(1)
        if len(y) > 1:
            coords = y[1]
    else :
        return [None, None, 'Tidak ditemukan', 1]

    if coords is None:
        lat = long = None
    else:
        lat = float(re.split('%|C', coords)[0])
        long = float(re.split('%|C', coords)[2])

    return [lat, long, name_, 0]

In [3]:
file_input = 'Kantor_ATRBPN.xlsx'
file_output = 'Kantor_ATRBPN.csv'

df = pd.read_excel('input/{}'.format(file_input))
df['nama_google'] = ''

In [4]:
df_na = df.loc[df.latitude.isna()]
df_no_na = df.loc[~df.latitude.isna()]
df_na['isna'] = 1
df_no_na['isna'] = 0

In [33]:
options = webdriver.ChromeOptions()
options.add_argument('--start-maximized')
options.add_argument('--disable-extensions')
options.add_experimental_option("useAutomationExtension", False)
options.add_experimental_option("excludeSwitches", ["enable-automation"])

driver = webdriver.Chrome(options=options)

for i, r in tqdm(df_na.iterrows(), total=df_na.shape[0]):
    if r['isna'] == 1 :
        location_name = r['Nama']
        df_na.loc[i, ['latitude', 'longitude', 'nama_google', 'isna']] = get_long_lat(location_name)

 90%|████████▉ | 517/577 [00:00<00:00, 16659.79it/s]


TypeError: quote_from_bytes() expected bytes

In [34]:
df_na

,No,Nama,latitude,longitude,nama_google,isna
0,1,Kantor Wilayah Badan Pertanahan Nasional Provi...,5.575483,95.355679,Kantor Wilayah BPN Provinsi Aceh,0
1,2,"Kantor Pertanahan Kota Banda Aceh, Provinsi Aceh",5.536943,95.338245,Kantor Pertanahan (BPN) Kota Banda Aceh,0
2,3,"Kantor Pertanahan Kota Sabang, Provinsi Aceh",5.890010,95.312413,Kantor Pertanahan Kota Sabang,0
3,4,"Kantor Pertanahan Kabupaten Aceh Besar, Provin...",5.297718,95.600019,Dinas Pertanahan Kabupaten Aceh Besar,0
4,5,"Kantor Pertanahan Kabupaten Aceh Barat, Provin...",4.152848,96.128994,Dinas Pertanahan Kabupaten Aceh Barat,0
...,...,...,...,...,...,...
572,573,NaN,NaN,NaN,,1
573,574,NaN,NaN,NaN,,1
574,575,NaN,NaN,NaN,,1
575,576,NaN,NaN,NaN,,1


In [35]:
x = pd.concat([df_na, df_no_na])
x = x.sort_index()
x.to_csv("output/{}".format(file_output), sep=';', index=False)